In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from sk import predict
from glob import glob
from tqdm import tqdm

In [3]:
import malaya
import malaya_speech
import tensorflow as tf
malaya.__version__, tf.__version__

2021-10-07 21:52:23.293288: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


('4.7', '2.5.0')

In [4]:
# !pip3 install tensorflow-text==2.5 --no-deps
# !pip3 install tensorflow_hub

In [5]:
# super_super_tiny = malaya.true_case.transformer(model = 'super-tiny-t5', device = 'cpu:0')

In [6]:
aligner = malaya_speech.force_alignment.deep_aligner(model = 'conformer-transducer', device = 'cpu:0')

2021-10-07 21:52:56.583742: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-07 21:52:56.587201: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-07 21:52:56.617822: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-07 21:52:56.617883: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2021-10-07 21:52:56.617896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2021-10-07 21:52:56.618381: I tensorflow/stream_executor/cuda/cuda_diagnos

In [8]:
wavs = glob('dari-pasentran-ke-istana-short/*/*.wav')
len(wavs)

20345

In [9]:
def put(alignment, length, min_threshold = 0.6, min_threshold_fullstop = 0.8):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [22]:
results = {}
for wav in tqdm(wavs):
    with open(wav.replace('-wav-', '-text-') + '.txt') as fopen:
        data = fopen.read().lower()
    y, sr = malaya_speech.load(wav)
    result = aligner.predict(y, data)
    if ' '.join([r['text'] for r in result['words_alignment']]) != data:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    results[wav] = ' '.join(o)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20345/20345 [44:10<00:00,  7.68it/s]


In [23]:
len(results), len(wavs)

(19529, 20345)

In [24]:
results

{'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/gus-dur-dan-islam-liberal-5.mp3-707.wav': 'new revivalism ,',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/di-ambang-perubahan-6.mp3-23.wav': 'beliau adalah salah seorang daripada beberapa kiai .',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/kaherah-baghdad-dan-eropah-3.mp3-302.wav': 'akhirnya , beliau berjaya menyertai ,',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/membesar-di-pesantran-dan-keluarga-1.mp3-454.wav': 'mereka tidak berfikir , atau ber ,',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/perubahan-rejim-dan-pergelutan-demi-kelangsungan-12.mp3-2101.wav': 'indon kampungan ,',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/islam-politik-dan-pilihanraya-10.mp3-1690.wav': 'gugur mempunyai hubungan baik dengan ulama-ulama di aceh .',
 'dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-

In [32]:
ipd.Audio('dari-pasentran-ke-istana-short/output-wav-dari-pasentran-ke-istana/pesantren-dan-pembaharuan-4.mp3-485.wav')

In [25]:
import json
with open('comma-fullstop-pasentran-ke-istana-short.json', 'w') as fopen:
    json.dump(results, fopen)